In [1]:
import json
import os
import numpy as np
import re
from tqdm import tqdm
from glob import glob
from collections import defaultdict
from datasets import Audio
import torchaudio
import torch

In [2]:
sr = 16000
reader = Audio(sampling_rate = sr)

In [3]:
with open('/home/husein/ssd3/semisupervised-whisper-stt/dataset-whisper-stt-v4.json') as fopen:
    data = json.load(fopen)

In [4]:
d = defaultdict(list)
for d_ in data:
    if d_['avg_logprob'] >= -0.3:
        k = f"{d_['no']}-{d_['i']}"
        d[k].append(d_)

In [5]:
def read(f):
    audio = reader.decode_example(reader.encode_example(f))['array']
    return audio, len(audio) / sr

In [6]:
!mkdir /home/husein/ssd3/postfilter-stt
!mkdir /home/husein/ssd3/postfilter-stt/part3

mkdir: cannot create directory ‘/home/husein/ssd3/postfilter-stt’: File exists
mkdir: cannot create directory ‘/home/husein/ssd3/postfilter-stt/part3’: File exists


In [7]:
maxlen = 12

index = 0
directory = '/home/husein/ssd3/postfilter-stt/part3'
X, Y = [], []

for key in tqdm(sorted(list(d.keys()))):
    values = {d_['k']: d_ for d_ in d[key]}
    sorted_k = sorted(values.keys())
    temp = []
    temp_text = []
    idx = []
    lens = 0
    last = None
    
    for k in sorted_k:
        y, l = read(values[k]['file'])
        y = y[int(0.1 * sr):-int(0.1 * sr)]
        t = values[k]['cleaned']

        if len(re.findall(r'\d+', t)):
            continue

        if len(t.strip()) < 2:
            continue

        if last is None:
            temp.append(y)
            idx.append(k)
            temp_text.append(t)
            lens += l

        elif k - last != 1 or (lens + l) >= 12:
            audio_path = os.path.join(directory, f'{index}.mp3')
            a = np.concatenate(temp, axis = 0)
            torchaudio.save(audio_path, 
                            torch.tensor(a.astype('float32')).unsqueeze(0), 
                            16000, format='mp3')
            X.append(audio_path)
            Y.append(' '.join(temp_text).strip())
            index += 1
            
            temp = [y]
            lens = l
            idx = [k]
            temp_text = [t]
        else:
            lens += l
            temp.append(y)
            idx.append(k)
            temp_text.append(t)

        last = k

    if len(temp):
        audio_path = os.path.join(directory, f'{index}.mp3')
        a = np.concatenate(temp, axis = 0)
        torchaudio.save(audio_path, 
                            torch.tensor(a.astype('float32')).unsqueeze(0), 
                            16000, format='mp3')
        X.append(audio_path)
        Y.append(' '.join(temp_text).strip())
        index += 1

 67%|███████████████████████▎           | 43758/65553 [1:25:12<29:00, 12.52it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
len(X)

151739

In [9]:
total_lens = []
for f in tqdm(X):
    audio = reader.decode_example(reader.encode_example(f))['array']
    total_lens.append(len(audio) / sr)

 84%|████████████████████████████▍     | 126814/151739 [05:12<00:59, 418.20it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
sum(total_lens) / 60 / 60

323.3796299999264

In [11]:
with open('postfilter-part3.json', 'w') as fopen:
    json.dump({'X': X, 'Y': Y}, fopen)